In [678]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics 
import math
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.metrics import mean_squared_error

In [679]:
df2=pd.read_csv("energy_dataset.csv")
df1=pd.read_csv("weather_features.csv")

df=pd.concat([df1, df2], axis=1, join='inner')

In [680]:
df

,dt_iso,city_name,temp,temp_min,temp_max,pressure,humidity,wind_speed,wind_deg,rain_1h,...,generation waste,generation wind offshore,generation wind onshore,forecast solar day ahead,forecast wind offshore eday ahead,forecast wind onshore day ahead,total load forecast,total load actual,price day ahead,price actual
0,2015-01-01 00:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,...,196.0,0.0,6378.0,17.0,NaN,6436.0,26118.0,25385.0,50.10,65.41
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,1001,77,1,62,0.0,...,195.0,0.0,5890.0,16.0,NaN,5856.0,24934.0,24382.0,48.10,64.92
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,196.0,0.0,5461.0,8.0,NaN,5454.0,23515.0,22734.0,47.33,64.48
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,191.0,0.0,5238.0,2.0,NaN,5151.0,22642.0,21286.0,42.27,59.32
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,1002,78,0,23,0.0,...,189.0,0.0,4935.0,9.0,NaN,4861.0,21785.0,20264.0,38.41,56.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,1028,71,3,300,0.0,...,277.0,0.0,3113.0,96.0,NaN,3253.0,30619.0,30653.0,68.85,77.02
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,1029,87,2,310,0.3,...,280.0,0.0,3288.0,51.0,NaN,3353.0,29932.0,29735.0,68.40,76.16
35061,2018-12-28 12:00:00+01:00,Valencia,284.150,284.150,284.150,1028,76,1,290,0.0,...,286.0,0.0,3503.0,36.0,NaN,3404.0,27903.0,28071.0,66.88,74.30
35062,2018-12-28 13:00:00+01:00,Valencia,285.660,285.150,286.150,1028,76,1,0,0.0,...,287.0,0.0,3586.0,29.0,NaN,3273.0,25450.0,25801.0,63.93,69.89


In [681]:
df.isnull().sum()

dt_iso                                             0
city_name                                          0
temp                                               0
temp_min                                           0
temp_max                                           0
pressure                                           0
humidity                                           0
wind_speed                                         0
wind_deg                                           0
rain_1h                                            0
rain_3h                                            0
snow_3h                                            0
clouds_all                                         0
weather_id                                         0
weather_main                                       0
weather_description                                0
weather_icon                                       0
time                                               0
generation biomass                            

In [682]:
def train_cats(df):
    for n,c in df.items():
        if is_string_dtype(c):
            df[n]=c.astype("category").cat.as_ordered()

In [683]:
train_cats(df)

In [684]:
df["city_name"].cat.codes
df["dt_iso"].cat.codes

0            0
1            1
2            2
3            3
4            4
         ...  
35059    34978
35060    34979
35061    34980
35062    34981
35063    34982
Length: 35064, dtype: int32

In [685]:
df3=df[["dt_iso","city_name","temp","temp_min","temp_max","humidity","price day ahead"]]

In [686]:
df=df3
df=df=df.sort_values(by="dt_iso",ascending=False)
df

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price day ahead
35063,2018-12-28 14:00:00+01:00,Valencia,286.660,286.150,287.150,71,64.27
35062,2018-12-28 13:00:00+01:00,Valencia,285.660,285.150,286.150,76,63.93
35061,2018-12-28 12:00:00+01:00,Valencia,284.150,284.150,284.150,76,66.88
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,87,68.40
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,71,68.85
...,...,...,...,...,...,...,...
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,78,38.41
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,78,42.27
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,78,47.33
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,77,48.10


In [687]:
df=df.rename({'price day ahead' : 'price'}, axis=1)

In [688]:
df

,dt_iso,city_name,temp,temp_min,temp_max,humidity,price
35063,2018-12-28 14:00:00+01:00,Valencia,286.660,286.150,287.150,71,64.27
35062,2018-12-28 13:00:00+01:00,Valencia,285.660,285.150,286.150,76,63.93
35061,2018-12-28 12:00:00+01:00,Valencia,284.150,284.150,284.150,76,66.88
35060,2018-12-28 11:00:00+01:00,Valencia,282.150,282.150,282.150,87,68.40
35059,2018-12-28 10:00:00+01:00,Valencia,282.140,281.150,283.150,71,68.85
...,...,...,...,...,...,...,...
4,2015-01-01 04:00:00+01:00,Valencia,269.686,269.686,269.686,78,38.41
3,2015-01-01 03:00:00+01:00,Valencia,269.686,269.686,269.686,78,42.27
2,2015-01-01 02:00:00+01:00,Valencia,269.686,269.686,269.686,78,47.33
1,2015-01-01 01:00:00+01:00,Valencia,270.475,270.475,270.475,77,48.10


In [689]:
#df["price"]=np.log(df.price)

In [690]:
def fix_missing(df,col,name,nan_dict,is_train):
    if is_numeric_dtype(col):
        if pd.isnull(col).sum():
            df[name+"_na"]=pd.isnull(col)
            nan_dict[name]=col.median()
            df[name]=col.fillna(nan_dict[name])
    else:
        if is_numeric_dtype(col):
            if name in nan_dict:
                df[name+"_na"]=pd.isnull(col)
                df[name]=col.fillna(nan_dict[name])
            else:
                df[name]=col.fillna(df[name].median())

In [691]:
def numericalize(df,col,name):
    if not is_numeric_dtype(col):
        df[name]=col.cat.codes+1

In [692]:
def proc_df(df,y_fld,nan_dict=None,is_train=True):
    df=df.copy()
    y=df[y_fld].values
    
    df.drop(y_fld,axis=1,inplace=True)
    
    if nan_dict is None:
        nan_dict={}
    for n,c in df.items():
        fix_missing(df,c,n,nan_dict,is_train)
        numericalize(df,c,n)
    if is_train:
        return df,y,nan_dict
    return df,y

In [693]:
def split_train_val(df,n):
    return df[:n].copy(),df[n:].copy()

In [694]:
n_valid=23000
n_train=len(df)-n_valid
raw_train,raw_valid=split_train_val(df,n_train)

In [695]:
x_train,y_train,nas=proc_df(raw_train,'price')
x_valid,y_valid=proc_df(raw_valid,'price',nan_dict=nas,is_train=False)

In [696]:
m=RandomForestRegressor(n_estimators=1,bootstrap=False,n_jobs=-1)
m.fit(x_train,y_train)
m.score(x_train,y_train)

0.9999310438376917

In [697]:
def rmsle(x:list, y:list) -> float:
    
    
    n = len(x)
    msle = np.mean([(np.log(y[i] + 1) - np.log(x[i] + 1)) ** 2.0 for i in range(n)])
    return np.sqrt(msle)
    
    

In [698]:
def print_score(m):
    print(f"RMSLE of valid set {rmsle(m.predict(x_valid),y_valid)}")
    print(f"R^2 of valid set {m.score(x_valid,y_valid)}")
    print(f"RMSLE of train set {rmsle(m.predict(x_train),y_train)}")
    print(f"R^2 of train set {m.score(x_train,y_train)}")

In [699]:
print_score(m)

RMSLE of valid set 0.4355229378078845
R^2 of valid set -0.13351278458021731
RMSLE of train set 0.0017666951354923396
R^2 of train set 0.9999310438376917


In [700]:
m=RandomForestRegressor(n_estimators=10,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

Wall time: 69.1 ms
RMSLE of valid set 0.431488085045481
R^2 of valid set -0.09160352948894301
RMSLE of train set 0.0017666951354923396
R^2 of train set 0.9999310438376917


In [702]:
m=RandomForestRegressor(n_estimators=100,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)

Wall time: 543 ms
RMSLE of valid set 0.4325182486012066
R^2 of valid set -0.10084645214164878
RMSLE of train set 0.0017666951354923396
R^2 of train set 0.9999310438376917


In [ ]:
m=RandomForestRegressor(n_estimators=1000,bootstrap=False,n_jobs=-1)
%time m.fit(x_train,y_train)
print_score(m)